In [ ]:
import cv2

cap = cv2.VideoCapture("C:\\Users\\chand\\Trad._CV\\KOHLI_COVER_DRIVE.mp4") 
cnt = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break 
    cnt += 1
    print(f"Processing frame {cnt}")
   
cap.release()

In [ ]:
import cv2
import matplotlib.pyplot as plt

cap = cv2.VideoCapture("C:\\Users\\chand\\Trad._CV\\KOHLI_COVER_DRIVE.mp4") 
ret, image = cap.read()  

roi = image[50:200, 100:300] 

plt.imshow(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))
plt.show()


In [ ]:
import cv2
import numpy as np
import imutils
import regex as re
import matplotlib.pyplot as plt
import os

os.makedirs('Frame', exist_ok=True)
os.makedirs('Frame_b', exist_ok=True)

video_path ="C:\\Users\\chand\\Trad._CV\\KOHLI_COVER_DRIVE.mp4"
cap = cv2.VideoCapture(video_path)
cnt = 0

if not cap.isOpened():
    print("Error opening video stream or file")

ret, first_frame = cap.read()

while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        roi = frame
        plt.imshow(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)) 
        plt.show()
        
        if cv2.waitKey(50) & 0xFF == ord('q'):
            break
        
        cv2.imwrite('Frame/' + str(cnt) + '.png', roi)
        cnt += 1
    else:
        break

cv2.destroyAllWindows()

frames = os.listdir('Frame/')
frames.sort(key=lambda f: int(re.sub('\\D', '', f)))

images = []
trajectory_points = []
radius_points = []
width_height = []

for frame_filename in frames:
    frame_path = os.path.join('Frame/', frame_filename)
    img = cv2.imread(frame_path)
    result = img.copy()


    image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)


    lower1 = np.array([0, 100, 0])
    upper1 = np.array([10, 255, 255])
    lower2 = np.array([160, 100, 20])
    upper2 = np.array([180, 255, 255])


    lower_mask = cv2.inRange(image, lower1, upper1)
    upper_mask = cv2.inRange(image, lower2, upper2)
    full_mask = lower_mask + upper_mask
    result = cv2.bitwise_and(result, result, mask=full_mask)


    contours, _ = cv2.findContours(full_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    detectedCircles = []
    image = img.copy()


    for i, c in enumerate(contours):
        blobArea = cv2.contourArea(c)
        blobPerimeter = cv2.arcLength(c, True)

        if blobPerimeter != 0:
            blobCircularity = (4 * 3.1416 * blobArea) / (blobPerimeter**2)
            minCircularity = 0.2
            minArea = 35

     
            (x, y), radius = cv2.minEnclosingCircle(c)
            center = (int(x), int(y))
            radius = int(radius)

      
            if blobCircularity > minCircularity and blobArea > minArea:
                detectedCircles.append([center, radius, blobArea, blobCircularity])


    if len(detectedCircles) != 0:
        largest_blob = max(detectedCircles, key=lambda x: x[2] * x[3])
        largest_center, largest_radius, largest_area, largest_circularity = largest_blob

    
        if largest_circularity < 0.4 and len(detectedCircles) > 1:
            remaining_blobs = [blob for blob in detectedCircles if blob != largest_blob]
            largest_circularity_blob = max(remaining_blobs, key=lambda x: x[2] * x[3])
            largest_center, largest_radius, largest_area, largest_circularity = largest_circularity_blob

        w, h = 4 * largest_radius, 4 * largest_radius
        color = (255, 0, 0)

        x, y = largest_center
        cv2.circle(image, largest_center, largest_radius, color, 2)
        cv2.rectangle(image, (x - w // 2, y - h // 2), (x + w // 2, y + h // 2), (0, 0, 255), 3)
        trajectory_points.append(largest_center)
        radius_points.append(largest_radius)
        width_height.append([w, h])
        images.append(image)


output_directory = 'Frame_b/'
for i in range(len(images)):
    processed_image = images[i]
    cv2.imwrite(os.path.join(output_directory, f'processed_frame_{i}.png'), processed_image)


frames = os.listdir('Frame_b/')
frames.sort(key=lambda f: int(re.sub('\D', '', f)))
frame_array = []

for i in range(len(frames)):
  
    img = cv2.imread('Frame_b/' + frames[i])
    height, width, layers = img.shape
    size = (width, height)
    frame_array.append(img)

out = cv2.VideoWriter('Cric_with_bounding_boxes_.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 20, size)
for i in range(len(frame_array)):
    out.write(frame_array[i])
out.release()
